# **Verzamelen van gebruiker gegevens**

In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
import pandas as pd
import os
import random
from datetime import datetime
import geocoder

# Voeg nieuwe gebruiker toe of haal bestaande ID op
def genereer_gebruiker_id(voornaam, tussenvoegsel, achternaam, gebruikers_df):
    bestaande = gebruikers_df[
        (gebruikers_df["Voornaam"].fillna('').str.lower() == voornaam.lower()) &
        (gebruikers_df["Tussenvoegsel"].fillna('').str.lower() == tussenvoegsel.lower()) &
        (gebruikers_df["Achternaam"].fillna('').str.lower() == achternaam.lower())
    ]
    if not bestaande.empty:
        return bestaande.iloc[0]["Gebruiker_ID"]
    else:
        while True:
            nieuwe_id = str(random.randint(100000, 999999))
            if nieuwe_id not in gebruikers_df["Gebruiker_ID"].astype(str).values:
                return nieuwe_id

# Bepaal locatie op basis van postcode
def bepaal_locatie_op_postcode(postcode):
    try:
        g = geocoder.arcgis(postcode + ", Nederland")
        if g.ok and g.latlng:
            lat, lng = g.latlng
            provincie = g.state if g.state else "Onbekend"
            return provincie, lat, lng
    except:
        pass
    return "Onbekend", None, None

# ---------- Tweede GUI ----------
def start_advies_gui(gebruikers_df):
    root = tk.Tk()
    root.title("Adviezen voor de tuin")

    opties = {
        "Huisdieren": ["Hond", "Kat", "Konijn", "Geen"],
        "Tuinstijl": ["Modern", "Klassiek", "Natuurlijk", "Onderhoudsvriendelijk"],
        "Onderhoudsniveau": ["Laag", "Gemiddeld", "Hoog"],
        "Allergieën": ["Pollen", "Bijengif", "Grassen", "Geen"]
    }

    entries = {}

    def maak_multiple_choice(row, label_text, opties_lijst):
        tk.Label(root, text=label_text + ":").grid(row=row, column=0, sticky="w", pady=2)
        vars_lijst = []
        for i, optie in enumerate(opties_lijst):
            var = tk.BooleanVar()
            chk = tk.Checkbutton(root, text=optie, variable=var)
            chk.grid(row=row+i, column=1, sticky="w")
            vars_lijst.append((optie, var))
        return row + len(opties_lijst), vars_lijst

    row = 0
    for label, keuzes in opties.items():
        row, var_lijst = maak_multiple_choice(row, label, keuzes)
        entries[label] = var_lijst

    # Budget label en entry
    tk.Label(root, text="Budget (€):").grid(row=row, column=0, sticky="w", pady=5)
    budget_entry = tk.Entry(root)
    budget_entry.grid(row=row, column=1, sticky="w", pady=5)
    row += 1

    def on_save():
        advies_dict = {}
        for label, var_lijst in entries.items():
            geselecteerd = [optie for optie, var in var_lijst if var.get()]
            advies_dict[label] = ', '.join(geselecteerd)

        budget = budget_entry.get().strip()
        try:
            advies_dict["Budget"] = float(budget) if budget else None
        except ValueError:
            messagebox.showerror("Fout", "Voer een geldig getal in voor het budget.")
            return

        for key, value in advies_dict.items():
            gebruikers_df.at[gebruikers_df.index[-1], key] = value

        gebruikers_df.to_csv("Geb_data.csv", index=False)
        print("✅ Adviesgegevens toegevoegd aan gebruiker:")
        print(gebruikers_df.tail(1))

        root.quit()
        root.destroy()

    tk.Button(root, text="Opslaan", command=on_save).grid(row=row, column=0, columnspan=2, pady=10)

    root.mainloop()

# ---------- Eerste GUI ----------
def start_gui():
    root = tk.Tk()
    root.title("Tuinadvies - Gebruikersgegevens")

    labels = ["Naam", "Geboortejaar", "Geslacht", "Ervaring met tuinieren (jaren)", "Soort woning", "Postcode"]
    entries = {}

    geslacht_opties = ["Man", "Vrouw", "Anders"]
    woning_opties = ["Appartement", "Tussenwoning", "Hoekwoning", "Vrijstaand"]

    # Naam
    tk.Label(root, text="Naam").grid(row=0, column=0, sticky="w", pady=3)
    naam_entry = tk.Entry(root)
    naam_entry.grid(row=0, column=1, sticky="w", pady=3)
    entries["Naam"] = naam_entry

    # Geboortejaar
    tk.Label(root, text="Geboortejaar").grid(row=1, column=0, sticky="w", pady=3)
    huidig_jaar = datetime.now().year
    jaren = list(range(huidig_jaar, 1899, -1))
    geboortejaar_combo = ttk.Combobox(root, values=jaren, state="readonly")
    geboortejaar_combo.current(0)
    geboortejaar_combo.grid(row=1, column=1, sticky="w", pady=3)
    entries["Geboortejaar"] = geboortejaar_combo

    # Ervaring
    tk.Label(root, text="Ervaring met tuinieren (jaren)").grid(row=2, column=0, sticky="w", pady=3)
    ervaring_entry = tk.Entry(root)
    ervaring_entry.grid(row=2, column=1, sticky="w", pady=3)
    entries["Ervaring met tuinieren (jaren)"] = ervaring_entry

    # Geslacht
    tk.Label(root, text="Geslacht:").grid(row=3, column=0, sticky="w", pady=3)
    geslacht_var = tk.StringVar(value=geslacht_opties[0])
    for idx, optie in enumerate(geslacht_opties):
        tk.Radiobutton(root, text=optie, variable=geslacht_var, value=optie).grid(row=3, column=1+idx, sticky="w")

    # Woning
    tk.Label(root, text="Soort woning:").grid(row=4, column=0, sticky="w", pady=3)
    woning_var = tk.StringVar(value=woning_opties[0])
    for idx, optie in enumerate(woning_opties):
        tk.Radiobutton(root, text=optie, variable=woning_var, value=optie).grid(row=4, column=1+idx, sticky="w")

    # Postcode
    tk.Label(root, text="Postcode:").grid(row=5, column=0, sticky="w", pady=3)
    postcode_entry = tk.Entry(root)
    postcode_entry.grid(row=5, column=1, sticky="w", pady=3)
    entries["Postcode"] = postcode_entry

    def split_naam_volledig(naam):
        delen = naam.strip().split()
        if len(delen) == 0:
            return "", "", ""
        elif len(delen) == 1:
            return delen[0], "", ""
        elif len(delen) == 2:
            return delen[0], "", delen[1]
        else:
            return delen[0], " ".join(delen[1:-1]), delen[-1]

    def on_next():
        data = {}
        for label in ["Naam", "Geboortejaar", "Ervaring met tuinieren (jaren)", "Postcode"]:
            waarde = entries[label].get().strip()
            if not waarde:
                messagebox.showerror("Fout", f"Vul alle velden in (ontbreekt: {label})")
                return
            data[label] = waarde

        try:
            geboortejaar = int(data["Geboortejaar"])
            huidig_jaar = datetime.now().year
            leeftijd = huidig_jaar - geboortejaar
            if leeftijd < 0 or leeftijd > 120:
                messagebox.showerror("Fout", "Voer een geldig geboortejaar in.")
                return
            data["Leeftijd"] = leeftijd
            data["Ervaring met tuinieren (jaren)"] = int(data["Ervaring met tuinieren (jaren)"])
        except ValueError:
            messagebox.showerror("Fout", "Geboortejaar en ervaring moeten getallen zijn.")
            return

        del data["Geboortejaar"]

        data["Geslacht"] = geslacht_var.get()
        data["Soort woning"] = woning_var.get()
        postcode = data.pop("Postcode")

        voornaam, tussenvoegsel, achternaam = split_naam_volledig(data["Naam"])
        data["Voornaam"] = voornaam.lower()
        data["Tussenvoegsel"] = tussenvoegsel.lower()
        data["Achternaam"] = achternaam.lower()
        del data["Naam"]

        bestand = "Geb_data.csv"
        kolommen = ["Tijdstip", "Gebruiker_ID", "Voornaam", "Tussenvoegsel", "Achternaam", "Leeftijd",
                    "Geslacht", "Provincie", "Latitude", "Longitude",
                    "Ervaring met tuinieren (jaren)", "Soort woning",
                    "Tuinstijl", "Onderhoudsniveau", "Allergieën", "Huisdieren", "Budget"]

        if os.path.exists(bestand):
            gebruikers_df = pd.read_csv(bestand)
        else:
            gebruikers_df = pd.DataFrame(columns=kolommen)

        gebruiker_id = genereer_gebruiker_id(voornaam, tussenvoegsel, achternaam, gebruikers_df)
        provincie, lat, lng = bepaal_locatie_op_postcode(postcode)

        data["Gebruiker_ID"] = gebruiker_id
        data["Tijdstip"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        data["Provincie"] = provincie
        data["Latitude"] = lat
        data["Longitude"] = lng

        for kolom in kolommen:
            if kolom not in data:
                data[kolom] = None

        gebruikers_df = pd.concat([gebruikers_df, pd.DataFrame([data])], ignore_index=True)
        gebruikers_df.to_csv(bestand, index=False)

        root.quit()
        root.destroy()

        start_advies_gui(gebruikers_df)

    tk.Button(root, text="Volgende", command=on_next).grid(row=6, column=0, columnspan=5, pady=10)

    root.mainloop()

# ---------- Start applicatie ----------
if __name__ == "__main__":
    start_gui()


Status code 403 from https://nominatim.openstreetmap.org/search: ERROR - 403 Client Error: Forbidden for url: https://nominatim.openstreetmap.org/search?q=51.90326569690521%2C+4.506409961966057&format=jsonv2&addressdetails=1&limit=1
C:\Users\admin_sven\AppData\Local\Temp\ipykernel_16584\2710861514.py:220: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gebruikers_df = pd.concat([gebruikers_df, pd.DataFrame([data])], ignore_index=True)


✅ Adviesgegevens toegevoegd aan gebruiker:
               Tijdstip  Gebruiker_ID Voornaam Tussenvoegsel Achternaam  \
10  2025-05-21 12:13:48        249568     sven                  satimin   

    Leeftijd Geslacht Provincie   Latitude  Longitude  \
10        23      Man  Onbekend  51.903266    4.50641   

    Ervaring met tuinieren (jaren) Soort woning Tuinstijl Onderhoudsniveau  \
10                              23  Appartement                              

   Allergieën Huisdieren  Budget  
10                           NaN  


In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
import pandas as pd
import os
import random
from datetime import datetime
import pgeocode  # ✅ Nieuw toegevoegd

# Functie om provincie op te halen via pgeocode
def bepaal_locatie_op_postcode(postcode):
    nomi = pgeocode.Nominatim('nl')
    locatie = nomi.query_postal_code(postcode)
    
    if locatie is not None and pd.notnull(locatie.admin_name1):
        provincie = locatie.admin_name1
        lat = locatie.latitude
        lng = locatie.longitude
        return provincie, lat, lng
    
    return "Onbekend", None, None

def genereer_gebruiker_id(voornaam, tussenvoegsel, achternaam, gebruikers_df):
    bestaande = gebruikers_df[
        (gebruikers_df["Voornaam"].fillna('').str.lower() == voornaam.lower()) &
        (gebruikers_df["Tussenvoegsel"].fillna('').str.lower() == tussenvoegsel.lower()) &
        (gebruikers_df["Achternaam"].fillna('').str.lower() == achternaam.lower())
    ]
    if not bestaande.empty:
        return bestaande.iloc[0]["Gebruiker_ID"]
    else:
        while True:
            nieuwe_id = str(random.randint(100000, 999999))
            if nieuwe_id not in gebruikers_df["Gebruiker_ID"].astype(str).values:
                return nieuwe_id

def start_advies_gui(gebruikers_df):
    root = tk.Tk()
    root.title("Adviezen voor de tuin")

    opties = {
        "Huisdieren": ["Hond", "Kat", "Konijn", "Geen"],
        "Tuinstijl": ["Modern", "Klassiek", "Natuurlijk", "Onderhoudsvriendelijk"],
        "Onderhoudsniveau": ["Laag", "Gemiddeld", "Hoog"],
        "Allergieën": ["Pollen", "Bijengif", "Grassen", "Geen"]
    }

    entries = {}

    def maak_multiple_choice(row, label_text, opties_lijst):
        tk.Label(root, text=label_text + ":").grid(row=row, column=0, sticky="w", pady=2)
        vars_lijst = []
        for i, optie in enumerate(opties_lijst):
            var = tk.BooleanVar()
            chk = tk.Checkbutton(root, text=optie, variable=var)
            chk.grid(row=row+i, column=1, sticky="w")
            vars_lijst.append((optie, var))
        return row + len(opties_lijst), vars_lijst

    row = 0
    for label, keuzes in opties.items():
        row, var_lijst = maak_multiple_choice(row, label, keuzes)
        entries[label] = var_lijst

    tk.Label(root, text="Budget (€):").grid(row=row, column=0, sticky="w", pady=5)
    budget_entry = tk.Entry(root)
    budget_entry.grid(row=row, column=1, sticky="w", pady=5)
    row += 1

    def on_save():
        advies_dict = {}
        for label, var_lijst in entries.items():
            geselecteerd = [optie for optie, var in var_lijst if var.get()]
            advies_dict[label] = ', '.join(geselecteerd)

        budget = budget_entry.get().strip()
        try:
            advies_dict["Budget"] = float(budget) if budget else None
        except ValueError:
            messagebox.showerror("Fout", "Voer een geldig getal in voor het budget.")
            return

        for key, value in advies_dict.items():
            gebruikers_df.at[gebruikers_df.index[-1], key] = value

        gebruikers_df.to_csv("Geb_data.csv", index=False)
        print("✅ Adviesgegevens toegevoegd aan gebruiker:")
        print(gebruikers_df.tail(1))

        root.quit()
        root.destroy()

    tk.Button(root, text="Opslaan", command=on_save).grid(row=row, column=0, columnspan=2, pady=10)

    root.mainloop()

def start_gui():
    root = tk.Tk()
    root.title("Tuinadvies - Gebruikersgegevens")

    labels = ["Naam", "Geboortejaar", "Geslacht", "Ervaring met tuinieren (jaren)", "Soort woning", "Postcode"]
    entries = {}

    geslacht_opties = ["Man", "Vrouw", "Anders"]
    woning_opties = ["Appartement", "Tussenwoning", "Hoekwoning", "Vrijstaand"]

    tk.Label(root, text="Naam").grid(row=0, column=0, sticky="w", pady=3)
    naam_entry = tk.Entry(root)
    naam_entry.grid(row=0, column=1, sticky="w", pady=3)
    entries["Naam"] = naam_entry

    tk.Label(root, text="Geboortejaar").grid(row=1, column=0, sticky="w", pady=3)
    huidig_jaar = datetime.now().year
    jaren = list(range(huidig_jaar, 1899, -1))
    geboortejaar_combo = ttk.Combobox(root, values=jaren, state="readonly")
    geboortejaar_combo.current(0)
    geboortejaar_combo.grid(row=1, column=1, sticky="w", pady=3)
    entries["Geboortejaar"] = geboortejaar_combo

    tk.Label(root, text="Ervaring met tuinieren (jaren)").grid(row=2, column=0, sticky="w", pady=3)
    ervaring_entry = tk.Entry(root)
    ervaring_entry.grid(row=2, column=1, sticky="w", pady=3)
    entries["Ervaring met tuinieren (jaren)"] = ervaring_entry

    tk.Label(root, text="Geslacht:").grid(row=3, column=0, sticky="w", pady=3)
    geslacht_var = tk.StringVar(value=geslacht_opties[0])
    for idx, optie in enumerate(geslacht_opties):
        tk.Radiobutton(root, text=optie, variable=geslacht_var, value=optie).grid(row=3, column=1+idx, sticky="w")

    tk.Label(root, text="Soort woning:").grid(row=4, column=0, sticky="w", pady=3)
    woning_var = tk.StringVar(value=woning_opties[0])
    for idx, optie in enumerate(woning_opties):
        tk.Radiobutton(root, text=optie, variable=woning_var, value=optie).grid(row=4, column=1+idx, sticky="w")

    tk.Label(root, text="Postcode:").grid(row=5, column=0, sticky="w", pady=3)
    postcode_entry = tk.Entry(root)
    postcode_entry.grid(row=5, column=1, sticky="w", pady=3)
    entries["Postcode"] = postcode_entry

    def split_naam_volledig(naam):
        delen = naam.strip().split()
        if len(delen) == 0:
            return "", "", ""
        elif len(delen) == 1:
            return delen[0], "", ""
        elif len(delen) == 2:
            return delen[0], "", delen[1]
        else:
            return delen[0], " ".join(delen[1:-1]), delen[-1]

    def on_next():
        data = {}
        for label in ["Naam", "Geboortejaar", "Ervaring met tuinieren (jaren)", "Postcode"]:
            waarde = entries[label].get().strip()
            if not waarde:
                messagebox.showerror("Fout", f"Vul alle velden in (ontbreekt: {label})")
                return
            data[label] = waarde

        try:
            geboortejaar = int(data["Geboortejaar"])
            huidig_jaar = datetime.now().year
            leeftijd = huidig_jaar - geboortejaar
            if leeftijd < 0 or leeftijd > 120:
                messagebox.showerror("Fout", "Voer een geldig geboortejaar in.")
                return
            data["Leeftijd"] = leeftijd
            data["Ervaring met tuinieren (jaren)"] = int(data["Ervaring met tuinieren (jaren)"])
        except ValueError:
            messagebox.showerror("Fout", "Geboortejaar en ervaring moeten getallen zijn.")
            return

        del data["Geboortejaar"]

        data["Geslacht"] = geslacht_var.get()
        data["Soort woning"] = woning_var.get()
        postcode = data.pop("Postcode")

        voornaam, tussenvoegsel, achternaam = split_naam_volledig(data["Naam"])
        data["Voornaam"] = voornaam.lower()
        data["Tussenvoegsel"] = tussenvoegsel.lower()
        data["Achternaam"] = achternaam.lower()
        del data["Naam"]

        bestand = "Geb_data.csv"
        kolommen = ["Tijdstip", "Gebruiker_ID", "Voornaam", "Tussenvoegsel", "Achternaam", "Leeftijd",
                    "Geslacht", "Provincie", "Latitude", "Longitude",
                    "Ervaring met tuinieren (jaren)", "Soort woning",
                    "Tuinstijl", "Onderhoudsniveau", "Allergieën", "Huisdieren", "Budget"]

        if os.path.exists(bestand):
            gebruikers_df = pd.read_csv(bestand)
        else:
            gebruikers_df = pd.DataFrame(columns=kolommen)

        gebruiker_id = genereer_gebruiker_id(voornaam, tussenvoegsel, achternaam, gebruikers_df)
        provincie, lat, lng = bepaal_locatie_op_postcode(postcode)

        data["Gebruiker_ID"] = gebruiker_id
        data["Tijdstip"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        data["Provincie"] = provincie
        data["Latitude"] = lat
        data["Longitude"] = lng

        for kolom in kolommen:
            if kolom not in data:
                data[kolom] = None

        gebruikers_df = pd.concat([gebruikers_df, pd.DataFrame([data])], ignore_index=True)

        start_advies_gui(gebruikers_df)

    tk.Button(root, text="Volgende", command=on_next).grid(row=6, column=0, columnspan=3, pady=10)

    root.mainloop()

if __name__ == "__main__":
    start_gui()


___
# **Verzamelen van locatie weeregegevens**

In [2]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from timezonefinder import TimezoneFinder
import pandas as pd
import numpy as np
from meteostat import Stations, Daily
from pvlib.location import Location
import ast

# ---------------------------
# 📍 Coördinaten splitsen uit kolom 'Coördinaten'
# ---------------------------
def splits_coordinaten(df):
    latitudes = []
    longitudes = []

    for coord in df['Coördinaten']:
        try:
            if isinstance(coord, str) and coord.startswith('['):
                coord = ast.literal_eval(coord)
            if isinstance(coord, (list, tuple)) and len(coord) == 2:
                latitudes.append(coord[0])
                longitudes.append(coord[1])
            else:
                latitudes.append(None)
                longitudes.append(None)
        except:
            latitudes.append(None)
            longitudes.append(None)

    df['Latitude'] = latitudes
    df['Longitude'] = longitudes
    return df

# Laad de gebruikersdata uit het CSV-bestand
gebruikers_df = pd.read_csv("Geb_data.csv")
# gebruikers_df = splits_coordinaten(gebruikers_df)

# 📍 Haal de coördinaten en gebruiker ID op uit de laatste rij
laatste_lat = gebruikers_df['Latitude'].iloc[-1]
laatste_lon = gebruikers_df['Longitude'].iloc[-1]
gebruikers_id = gebruikers_df['Gebruiker_ID'].iloc[-1]  # Pas aan als kolom anders heet

# 🌍 Zoek automatisch de tijdzone op
tf = TimezoneFinder()
plaatsnaam = tf.timezone_at(lat=laatste_lat, lng=laatste_lon)

# 🕒 Definieer start- en einddatum (start op 1e dag 3 jaar geleden)
end = datetime.now()
start = (end - relativedelta(years=3)).replace(day=1)

# 🌤️ Dagelijkse weerdata ophalen
station = Stations().nearby(laatste_lat, laatste_lon).fetch(1)
station_id = station.index[0]
data = Daily(station_id, start, end).fetch()

df_dag = data[['tavg', 'prcp', 'wspd']]
df_dag.index = pd.to_datetime(df_dag.index)

# 🌞 Zonhoogte per dag
locatie = Location(laatste_lat, laatste_lon, plaatsnaam)
zon_hoogtes = []
zonsopkomst_tijden = []
zonsondergang_tijden = []

for datum in df_dag.index:
    uren = pd.date_range(datum.strftime('%Y-%m-%d 00:00:00'), periods=24, freq='H', tz=plaatsnaam)
    zonne_positie = locatie.get_solarposition(uren)
    zonhoogte_dag = zonne_positie['apparent_elevation'].mean()
    zon_hoogtes.append(zonhoogte_dag)

    # Haal zonsopkomst en zonsondergang
    try:
        zonsopkomst = zonne_positie[zonne_positie['apparent_elevation'] > 0].index[0]
        zonsondergang = zonne_positie[zonne_positie['apparent_elevation'] > 0].index[-1]
        zonsopkomst_tijden.append(zonsopkomst.strftime('%H:%M'))
        zonsondergang_tijden.append(zonsondergang.strftime('%H:%M'))
    except IndexError:
        zonsopkomst_tijden.append("00:00")
        zonsondergang_tijden.append("00:00")

df_dag['zonhoogte (°)'] = zon_hoogtes
df_dag['zonsopkomst'] = zonsopkomst_tijden
df_dag['zonsondergang'] = zonsondergang_tijden

# 🥵 Hitte-index berekenen
def calculate_heat_index(temp_c):
    return temp_c * 1.8 + 32  # °C naar °F

df_dag['hitte_index (°F)'] = df_dag['tavg'].apply(calculate_heat_index).round(2)

# Kolommen hernoemen
df_dag = df_dag.rename(columns={
    'tavg': 'temp (°C)',
    'prcp': 'neerslag (mm)',
    'wspd': 'windsnelheid (m/s)',
})

# 🌤️ Daglengte berekenen in uren
df_dag['zonsondergang_dt'] = pd.to_datetime(df_dag['zonsondergang'], format='%H:%M', errors='coerce')
df_dag['zonsopkomst_dt'] = pd.to_datetime(df_dag['zonsopkomst'], format='%H:%M', errors='coerce')

df_dag['zonuren'] = (df_dag['zonsondergang_dt'] - df_dag['zonsopkomst_dt']).dt.total_seconds() / 3600

# Opschonen
df_dag = df_dag.drop(columns=['zonsondergang', 'zonsopkomst', 'zonsondergang_dt', 'zonsopkomst_dt'])
df_dag = df_dag.fillna(0)

# 📆 Maandgemiddelden berekenen
df_dag.index = pd.to_datetime(df_dag.index)
df_maand = df_dag.resample('M').mean().round(2)

# ➕ Gebruiker ID toevoegen en vooraan zetten
df_maand['Gebruiker_ID'] = gebruikers_id
cols = ['Gebruiker_ID'] + [c for c in df_maand.columns if c != 'Gebruiker_ID']
df_maand = df_maand[cols]

# Opslaan naar CSV met de naam ID van de gebruiker
df_maand.to_csv(f'{gebruikers_id}_maandgemiddelden.csv', index=True)

# ✅ Resultaat
df_maand

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

,Gebruiker_ID,temp (°C),neerslag (mm),windsnelheid (m/s),zonhoogte (°),hitte_index (°F),zonuren
time,,,,,,,
2022-05-31,249568,14.42,1.71,13.34,16.78,57.95,14.73
2022-06-30,249568,17.23,2.77,12.74,20.28,63.01,15.60
2022-07-31,249568,18.91,0.35,11.71,18.56,66.03,15.06
2022-08-31,249568,20.21,0.76,10.50,11.96,68.38,13.52
2022-09-30,249568,15.43,5.55,10.92,2.45,59.77,11.53
2022-10-31,249568,13.81,1.99,14.03,-7.78,56.86,9.58
2022-11-30,249568,9.03,4.05,17.43,-16.18,48.25,7.67
2022-12-31,249568,4.12,2.97,15.55,-20.12,39.42,7.00
2023-01-31,249568,6.06,4.79,20.51,-18.11,42.90,7.48


___
# **Planten dataframe**

In [12]:
import pandas as pd

data = {
    # Basisinfo (50 planten)
    "plant": [
        "Gewone margriet", "Rode klaver", "Wilde marjolein", "Grote kaardebol", "Wilde cichorei",
        "Kleine bevernel", "Zwarte toorts", "Gele morgenster", "Grote kattenstaart", "Dagkoekoeksbloem",
        "Moerasandoorn", "Echte valeriaan", "Knikkende distel", "Duizendguldenkruid", "Veldsalie",
        "Echte koekoeksbloem", "Kruipend zenegroen", "Vlasbekje", "Zomerfijnstraal", "Beemdkroon",
        "Gewone ossentong", "Echte sleutelbloem", "Bosandoorn", "Wilde reseda", "Blaassilene",
        "Echte guldenroede", "Klein streepzaad", "Hennepnetel", "Sint-janskruid", "Wilde bertram",
        "Veldbeemdgras", "Pinksterbloem", "Zevenblad", "Look-zonder-look", "Gewone dotterbloem",
        "Gevlekte aronskelk", "Bosviooltje", "Klimopereprijs", "Veldzuring", "Zandblauwtje",
        "Muskuskruid", "Ruw vergeet-mij-nietje", "Moerasvergeet-mij-nietje", "Kruipwilg", "Kleine ratelaar",
        "Wilde narcis", "Zandraket", "Boerenwormkruid", "Echte kamille", "Egelboterbloem"
    ],
    "scientificname": [
        "Leucanthemum vulgare", "Trifolium pratense", "Origanum vulgare", "Dipsacus fullonum", "Cichorium intybus",
        "Pimpinella saxifraga", "Verbascum nigrum", "Tragopogon pratensis", "Lythrum salicaria", "Silene dioica",
        "Stachys palustris", "Valeriana officinalis", "Carduus nutans", "Centaurium erythraea", "Salvia pratensis",
        "Lychnis flos-cuculi", "Ajuga reptans", "Linaria vulgaris", "Erigeron annuus", "Knautia arvensis",
        "Anchusa officinalis", "Primula veris", "Stachys sylvatica", "Reseda lutea", "Silene vulgaris",
        "Solidago virgaurea", "Crepis capillaris", "Galeopsis tetrahit", "Hypericum perforatum", "Achillea ptarmica",
        "Poa pratensis", "Cardamine pratensis", "Aegopodium podagraria", "Alliaria petiolata", "Caltha palustris",
        "Arum maculatum", "Viola reichenbachiana", "Veronica hederifolia", "Rumex acetosa", "Jasione montana",
        "Adoxa moschatellina", "Myosotis ramosissima", "Myosotis scorpioides", "Salix repens", "Rhinanthus minor",
        "Narcissus pseudonarcissus", "Arabidopsis thaliana", "Tanacetum vulgare", "Matricaria chamomilla", "Ranunculus flammula"
    ],
    
    # Numerieke waarden
    "min_hoogte_cm": [30, 20, 30, 100, 30, 15, 50, 30, 60, 30, 40, 50, 50, 10, 30, 30, 10, 30, 30, 30, 30, 10, 30, 30, 20, 30, 20, 30, 30, 30, 20, 15, 30, 30, 20, 20, 10, 10, 20, 15, 5, 10, 20, 10, 10, 15, 10, 30, 20, 15],
    "max_hoogte_cm": [60, 50, 60, 200, 120, 50, 150, 60, 150, 80, 100, 150, 150, 40, 60, 80, 30, 80, 100, 80, 80, 30, 100, 60, 60, 100, 60, 80, 80, 60, 100, 40, 100, 100, 50, 50, 20, 30, 60, 40, 15, 30, 50, 50, 40, 40, 50, 120, 50, 40],
    "uren_zon_per_dag": [8, 8, 8, 8, 10, 6, 10, 8, 6, 6, 4, 6, 8, 8, 8, 6, 4, 8, 8, 8, 8, 4, 4, 8, 8, 6, 8, 6, 8, 6, 8, 6, 4, 6, 4, 4, 4, 4, 8, 8, 4, 6, 4, 6, 8, 6, 8, 8, 8, 4],
    "waterbehoefte": [3, 3, 2, 3, 2, 2, 2, 3, 5, 3, 5, 4, 3, 3, 2, 4, 4, 2, 2, 3, 3, 4, 4, 2, 2, 3, 2, 3, 2, 3, 3, 4, 4, 3, 5, 4, 3, 3, 3, 2, 4, 4, 5, 4, 3, 4, 2, 2, 3, 5],
    "onderhoud_aandacht": [1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 3, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 2, 1, 1, 1, 1, 1, 2, 3, 2, 1, 2, 1, 1, 1, 3],
    
    # Ecologie & insecten
    "trekt_bijen_aan": ["Ja"]*25 + ["Nee"]*5 + ["Ja"]*20,
    "trekt_vlinders_aan": ["Ja"]*30 + ["Nee"]*10 + ["Soms"]*10,
    "waardplant_voor_rupsen": ["Nee"]*15 + ["Ja"]*20 + ["Nee"]*15,
    "giftig_voor_dieren": ["Nee"]*40 + ["Ja"]*5 + ["Nee"]*5,
    
    # Extra kenmerken
    "bloeitijd": [
        "Mei-sep", "Mei-okt", "Jun-sep", "Jul-aug", "Jul-sep",
        "Jun-aug", "Jun-aug", "Mei-jul", "Jun-aug", "Mei-jul",
        "Jun-aug", "Jun-aug", "Jun-aug", "Jun-aug", "Mei-jul",
        "Mei-jul", "Mei-jun", "Jun-sep", "Jun-okt", "Jun-sep",
        "Mei-jul", "Apr-jun", "Jun-aug", "Jun-sep", "Mei-aug",
        "Jul-sep", "Jun-okt", "Jun-sep", "Jun-aug", "Jun-aug",
        "Mei-jun", "Apr-jun", "Mei-jul", "Apr-jun", "Apr-jun",
        "Apr-mei", "Apr-jun", "Apr-jun", "Mei-jul", "Jun-aug",
        "Apr-mei", "Apr-jun", "Mei-sep", "Apr-mei", "Jun-aug",
        "Mrt-apr", "Apr-mei", "Jul-sep", "Jun-aug", "Mei-aug"
    ],
    "habitat": [
        "Grasland", "Grasland", "Droog grasland", "Ruderaal", "Droog grasland",
        "Droog grasland", "Ruderaal", "Grasland", "Moeras", "Bosrand",
        "Moeras", "Vochtige grond", "Ruderaal", "Grasland", "Droog grasland",
        "Vochtige grasland", "Bosrand", "Ruderaal", "Ruderaal", "Grasland",
        "Grasland", "Grasland", "Bos", "Ruderaal", "Grasland",
        "Bosrand", "Grasland", "Ruderaal", "Droog grasland", "Vochtige grasland",
        "Grasland", "Vochtige grasland", "Bosrand", "Bosrand", "Moeras",
        "Bos", "Bos", "Akker", "Grasland", "Droog grasland",
        "Bos", "Grasland", "Moeras", "Duin", "Grasland",
        "Bos", "Ruderaal", "Ruderaal", "Grasland", "Moeras"
    ],
    
    # Prompt-engineering
    "trefwoorden": [
        "bijenplant, bloemweide", "bodemverbeteraar, bijenplant", "keukenkruid, geur", "distel, insecten", "blauwe bloem, droogte",
        "schermbloem, droogte", "toorts, vlinders", "composiet, zaadverspreiding", "moerasplant, bijen", "roze bloem, schaduw",
        "moeras, vlinders", "geneeskrachtig, geur", "distel, vlinders", "roze bloem, grasland", "salie, bijen",
        "roze bloem, vochtig", "kruipend, bodembedekker", "vlinderbloem, droogte", "invasief, ruderaal", "paarse bloem, grasland",
        "blauwe bloem, grasland", "sleutelbloem, voorjaar", "bos, schaduw", "geel, ruderaal", "witte bloem, grasland",
        "guldenroede, laatbloeier", "composiet, grasland", "netel, bijen", "gele bloem, geneeskrachtig", "witte bloem, vochtig",
        "gras, grasland", "roze bloem, vochtig", "woekeraar, schaduw", "wit, bosrand", "gele bloem, moeras",
        "aronskelk, giftig", "viooltje, bos", "ereprijs, akker", "zuring, grasland", "blauwe bloem, droogte",
        "groen, bos", "blauwe bloem, grasland", "blauwe bloem, moeras", "wilg, duin", "ratelaar, halfparasiet",
        "narcis, voorjaar", "onkruid, ruderaal", "gele bloem, insectenwerend", "kamille, geur", "boterbloem, moeras"
    ]
}

df_inheems = pd.DataFrame(data)

# Toon de eerste 5 rijen
df_inheems.to_csv('inheemse_planten_50.csv', index=False)

In [13]:
# open de CSV-besnad inheemse_planten.csv
df_inheemse_planten = pd.read_csv("inheemse_planten_50.csv")
df_inheemse_planten

,plant,scientificname,min_hoogte_cm,max_hoogte_cm,uren_zon_per_dag,waterbehoefte,onderhoud_aandacht,trekt_bijen_aan,trekt_vlinders_aan,waardplant_voor_rupsen,giftig_voor_dieren,bloeitijd,habitat,trefwoorden
0,Gewone margriet,Leucanthemum vulgare,30,60,8,3,1,Ja,Ja,Nee,Nee,Mei-sep,Grasland,"bijenplant, bloemweide"
1,Rode klaver,Trifolium pratense,20,50,8,3,1,Ja,Ja,Nee,Nee,Mei-okt,Grasland,"bodemverbeteraar, bijenplant"
2,Wilde marjolein,Origanum vulgare,30,60,8,2,1,Ja,Ja,Nee,Nee,Jun-sep,Droog grasland,"keukenkruid, geur"
3,Grote kaardebol,Dipsacus fullonum,100,200,8,3,2,Ja,Ja,Nee,Nee,Jul-aug,Ruderaal,"distel, insecten"
4,Wilde cichorei,Cichorium intybus,30,120,10,2,1,Ja,Ja,Nee,Nee,Jul-sep,Droog grasland,"blauwe bloem, droogte"
5,Kleine bevernel,Pimpinella saxifraga,15,50,6,2,1,Ja,Ja,Nee,Nee,Jun-aug,Droog grasland,"schermbloem, droogte"
6,Zwarte toorts,Verbascum nigrum,50,150,10,2,1,Ja,Ja,Nee,Nee,Jun-aug,Ruderaal,"toorts, vlinders"
7,Gele morgenster,Tragopogon pratensis,30,60,8,3,1,Ja,Ja,Nee,Nee,Mei-jul,Grasland,"composiet, zaadverspreiding"
8,Grote kattenstaart,Lythrum salicaria,60,150,6,5,2,Ja,Ja,Nee,Nee,Jun-aug,Moeras,"moerasplant, bijen"
9,Dagkoekoeksbloem,Silene dioica,30,80,6,3,2,Ja,Ja,Nee,Nee,Mei-jul,Bosrand,"roze bloem, schaduw"


____
# **Synthetische Tuinen**

In [27]:
import pandas as pd
import random

# Laad de planten-DataFrame
df = pd.read_csv("inheemse_planten_50.csv")

# Tuinstijlen
tuinstijlen = ["Modern", "Klassiek", "Natuurlijk", "Onderhoudsvriendelijk"]

# Functie om planten te kiezen per stijl
def kies_planten(df, stijl):
    if stijl == "Modern":
        selectie = df[df["trefwoorden"].str.contains("geur|bloem", case=False, na=False)]
        n = min(6, len(selectie))
        return selectie.sample(n)
    elif stijl == "Klassiek":
        selectie = df[df["bloeitijd"].str.contains("mei|juni", case=False, na=False)]
        n = min(6, len(selectie))
        return selectie.sample(n)
    elif stijl == "Natuurlijk":
        selectie = df[df["trekt_bijen_aan"] == "Ja"]
        n = min(7, len(selectie))
        return selectie.sample(n)
    elif stijl == "Onderhoudsvriendelijk":
        selectie = df[df["onderhoud_aandacht"] <= 2]
        n = min(5, len(selectie))
        return selectie.sample(n)
    else:
        return df.sample(6)

# Functie om aantrekkingskracht te bepalen
def bepaal_aantrekkingskracht(planten_df):
    aantrekking = set()
    for i, rij in planten_df.iterrows():
        if "vlinder" in str(rij.get("trefwoorden", "")).lower():
            aantrekking.add("vlinders")
        if str(rij.get("trekt_bijen_aan", "")).lower() == "ja":
            aantrekking.add("bijen")
        if "vogel" in str(rij.get("trefwoorden", "")).lower():
            aantrekking.add("vogels")
    return ", ".join(sorted(aantrekking)) if aantrekking else "geen specifieke dieren"

# Functie om tuin bio te genereren
def genereer_bio(stijl, aantrekking):
    basis = f"Deze tuin is ontworpen in een {stijl.lower()} stijl."
    if "geen" in aantrekking:
        extra = " Ideaal voor rust en onderhoudsgemak, zonder veel fauna."
    else:
        extra = f" Deze tuin trekt vooral {aantrekking} aan en draagt bij aan een levendige tuinomgeving."
    return basis + extra

# Genereer 20 tuinen
tuinen_data = []

for i in range(1, 21):
    stijl = random.choice(tuinstijlen)
    planten_df = kies_planten(df, stijl)
    planten_namen = planten_df["plant"].tolist()
    biodiversiteit = planten_df["plant"].nunique()
    aantrekking = bepaal_aantrekkingskracht(planten_df)
    tuin_bio = genereer_bio(stijl, aantrekking)

    tuinen_data.append({
        "tuin_naam": f"Tuin_{i}",
        "stijl": stijl,
        "planten": ", ".join(planten_namen),
        "biodiversiteit_score": biodiversiteit,
        "aantrekkingskracht": aantrekking,
        "tuin_bio": tuin_bio
    })

# Zet om naar DataFrame
tuinen_df = pd.DataFrame(tuinen_data)

# id kolom toevoegen met een lengte van 6 random cijfers
tuinen_df["id"] = [str(random.randint(100000, 999999)) for _ in range(len(tuinen_df))]

# verplaats de id kolomn naar de eerste kolom
tuinen_df = tuinen_df[["id"] + [col for col in tuinen_df.columns if col != "id"]]

# Sla op naar CSV
tuinen_df.to_csv("tuinen.csv", index=False)

tuinen_df  

,id,tuin_naam,stijl,planten,biodiversiteit_score,aantrekkingskracht,tuin_bio
0,601690,Tuin_1,Klassiek,"Veldsalie, Blaassilene, Gewone ossentong, Moer...",6,bijen,Deze tuin is ontworpen in een klassiek stijl. ...
1,432461,Tuin_2,Natuurlijk,"Knikkende distel, Dagkoekoeksbloem, Gewone oss...",7,"bijen, vlinders",Deze tuin is ontworpen in een natuurlijk stijl...
2,419944,Tuin_3,Modern,"Sint-janskruid, Egelboterbloem, Blaassilene, B...",6,bijen,Deze tuin is ontworpen in een modern stijl. De...
3,546551,Tuin_4,Natuurlijk,"Wilde marjolein, Kleine ratelaar, Moerasandoor...",7,"bijen, vlinders",Deze tuin is ontworpen in een natuurlijk stijl...
4,409285,Tuin_5,Klassiek,"Echte koekoeksbloem, Zevenblad, Muskuskruid, K...",6,bijen,Deze tuin is ontworpen in een klassiek stijl. ...
5,874377,Tuin_6,Natuurlijk,"Boerenwormkruid, Echte sleutelbloem, Dagkoekoe...",7,bijen,Deze tuin is ontworpen in een natuurlijk stijl...
6,953535,Tuin_7,Klassiek,"Moerasvergeet-mij-nietje, Dagkoekoeksbloem, Za...",6,bijen,Deze tuin is ontworpen in een klassiek stijl. ...
7,261648,Tuin_8,Onderhoudsvriendelijk,"Vlasbekje, Grote kaardebol, Wilde narcis, Gele...",5,"bijen, vlinders",Deze tuin is ontworpen in een onderhoudsvriend...
8,327537,Tuin_9,Onderhoudsvriendelijk,"Grote kaardebol, Veldsalie, Echte koekoeksbloe...",5,bijen,Deze tuin is ontworpen in een onderhoudsvriend...
9,565240,Tuin_10,Modern,"Echte valeriaan, Beemdkroon, Vlasbekje, Sint-j...",6,"bijen, vlinders",Deze tuin is ontworpen in een modern stijl. De...


In [23]:
tuinen_df.columns

Index(['tuin_naam', 'stijl', 'planten', 'biodiversiteit_score',
       'aantrekkingskracht', 'tuin_bio'],
      dtype='object')

___
# **Advies_data**

- Data van het weer ✅
- Data van de gebruiker wat die wilt ✅
- Data van de soort planten die de gebruiker zou willen
<br><br>

**To do:**
- Data verzamelen va de planten met hun overlevings situatie
- Kijken wat nutig is voor de promt engineering
- Planten database gaan zoeken
- Kijken welk model het best is voor het opstellen van adviezen

___
# **Advies_data**

- Data van het weer ✅
- Data van de gebruiker wat die wilt ✅
- Data van de soort planten die de gebruiker zou willen
<br><br>

**To do:**
- Data verzamelen va de planten met hun overlevings situatie
- Kijken wat nutig is voor de promt engineering
- Planten database gaan zoeken
- Kijken welk model het best is voor het opstellen van adviezen

In [14]:
advies_df = pd.read_csv("Geb_data.csv")
advies_df = advies_df[advies_df['Gebruiker_ID'] == gebruikers_id]

# De laatse rij van advies_df
advies_df = advies_df.tail(1)

advies_df = advies_df[['Gebruiker_ID', 'Provincie', 'Latitude', 'Longitude', 
                       'Soort woning', 'Tuinstijl', 'Onderhoudsniveau', 'Allergieën',
                       'Huisdieren', 'Budget']]

advies_df

,Gebruiker_ID,Provincie,Latitude,Longitude,Soort woning,Tuinstijl,Onderhoudsniveau,Allergieën,Huisdieren,Budget
8,249568,South Holland,51.9225,4.4792,Appartement,Klassiek,Gemiddeld,Bijengif,Hond,NaN


In [6]:
print(f'df_maand columns: {df_maand.columns} & advies_df columns: {advies_df.columns}')
print(f'shape df_maand: {advies_df.shape} & shape advies_df: {df_maand.shape}')

df_maand columns: Index(['Gebruiker_ID', 'temp (°C)', 'neerslag (mm)', 'windsnelheid (m/s)',
       'zonhoogte (°)', 'hitte_index (°F)', 'zonuren'],
      dtype='object') & advies_df columns: Index(['Ervaring met tuinieren (jaren)', 'Soort woning', 'Tuinstijl',
       'Onderhoudsniveau', 'Allergieën', 'Huisdieren', 'Budget'],
      dtype='object')
shape df_maand: (1, 7) & shape advies_df: (37, 7)
